In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
data = pd.read_csv("../data/train.csv")
data

,Unnamed: 0,id,target_chr,target_start,target_end,target_strand,target_sequence,target_context,target_geneid,grna_target_chr,grna_target_start,grna_target_end,grna_target_strand,grna_target_sequence,genome,cell_line,cleavage_freq,epigen_ctcf,epigen_dnase,epigen_rrbs,epigen_h3k4me3,epigen_drip,energy_1,energy_2,energy_3,energy_4,energy_5,study_name,whole_genome,delivery_mode
0,5077,5077,chr11,32094683.0,32094705.0,+,ACCCCCCCCAACCCCGCCTCGGC,CTAAGTACCCAGATATCAAGGGCCTCCAGGTTCTGTTAAAGAGTTT...,RP1-65P5.6,chr6,43738556.0,43738578.0,-,GACCCCCTCCACCCCGCCTCCGG,hg19,U2OS,1.560000e-04,0.139,0.0,0.0,0.000,0.0,26.355,0.000000,8.809075,0.0000,26.355,Tsai_circle,1,0
1,8778,8778,chr11,22649986.0,22650008.0,-,AAGAGGAGGGAGATTGTTCCTGG,GCAGCCACTTGGGTGGAACTGGAGGCCATTATTCTAAGTGAAGTAA...,GAS2,chr6,43737291.0,43737313.0,-,GGGTGGGGGGAGTTTGCTCCTGG,hg19,U2OS,2.730000e-04,0.011,0.0,0.0,0.000,0.0,12.735,-4.939260,-4.939260,16.3350,16.335,Tsai_circle,1,0
2,25209,25209,NaN,NaN,NaN,+,GTGATAAGTGGAATTGCCATGTGAG,NaN,NaN,NaN,NaN,NaN,+,GTGATAAGTGGAATGCCATGTGG,NaN,NaN,9.213702e-06,0.000,0.0,0.0,0.000,0.0,-12.695,-42.815277,-47.572529,-9.1755,-10.195,Finkelstein,0,2
3,1750,1750,chr15,77121493.0,77121515.0,+,AGCACTGTGGATGGAGTTGGAGG,AAGGCTAAGAAGAAAAGATACAGATACAGATGAAGAAACGATGGCT...,SCAPER,chr20,31349756.0,31349778.0,+,GGCACTGCGGCTGGAGGTGGGGG,hg19,HEK293,3.302000e-03,0.000,0.0,0.0,0.014,0.0,17.605,5.982632,5.982632,21.1050,21.105,Tsai_circle,1,0
4,24783,24783,NaN,NaN,NaN,+,GTGATAAGATGGAATGCCATGTGGG,NaN,NaN,NaN,NaN,NaN,+,GTGATAAGTGGAATGCCATGTGG,NaN,NaN,4.029809e-04,0.000,0.0,0.0,0.000,0.0,-13.315,-48.090126,-48.090126,-10.8150,-10.815,Finkelstein,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20500,22598,22598,NaN,NaN,NaN,+,GTGGATAAGTGGAACTGCCATGTGG,NaN,NaN,NaN,NaN,NaN,+,GTGATAAGTGGAATGCCATGTGG,NaN,NaN,1.935917e-05,0.000,0.0,0.0,0.000,0.0,-2.095,-26.843519,-26.843519,0.4050,0.405,Finkelstein,0,2
20501,13984,13984,NaN,NaN,NaN,+,GACGCATAAAGATGAGACGCTTC,NaN,NaN,NaN,NaN,NaN,+,GACGCATAAAGATGAGACGCTGG,NaN,NaN,5.672865e-05,0.000,0.0,0.0,0.000,0.0,28.700,0.000000,21.484870,0.0000,28.700,Finkelstein,0,2
20502,22846,22846,NaN,NaN,NaN,+,GTGATAAAGTGGAATCGCCATGTGG,NaN,NaN,NaN,NaN,NaN,+,GTGATAAGTGGAATGCCATGTGG,NaN,NaN,6.571826e-19,0.000,0.0,0.0,0.000,0.0,-3.700,-29.040044,-29.040044,-1.2000,-1.200,Finkelstein,0,2
20503,6433,6433,chr7,123175079.0,123175101.0,-,TCTCCCCGCCCCCTCGCCTCTGG,TAACAAAAGTTCTTCTAAGCAGAATTATTTGATAGCTACTATCTCC...,IQUB,chr6,43738556.0,43738578.0,-,GACCCCCTCCACCCCGCCTCCGG,hg19,U2OS,2.800000e-05,0.000,0.0,0.0,0.994,0.0,17.935,-6.524068,-6.524068,17.9350,17.935,Tsai_circle,1,0


In [4]:
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [5]:
# Drop NA
params = ["grna_target_sequence", "target_sequence"]
params.append("cleavage_freq")
for col in params:
    data = data[data[col].notna()]
params.remove("cleavage_freq")

In [6]:
df_X = data.drop(columns = ["cleavage_freq"])
df_X = data.drop(columns = ["id", "target_chr","target_start", "target_end", "target_context", "target_geneid", "grna_target_chr", "grna_target_start", "grna_target_end", "genome", "cell_line", "epigen_ctcf", "epigen_dnase", "epigen_rrbs", "epigen_h3k4me3", "epigen_drip"])
df_X.dtypes

Unnamed: 0                int64
target_strand            object
target_sequence          object
grna_target_strand       object
grna_target_sequence     object
cleavage_freq           float64
energy_1                float64
energy_2                float64
energy_3                float64
energy_4                float64
energy_5                float64
study_name               object
whole_genome              int64
delivery_mode             int64
dtype: object

In [7]:
# Convert to string
df_X[params] = df_X[params].convert_dtypes()

# Convert strands + = 1, - = 0
def symbol_mapping(sym):
    mapping = {'+': 1.00, '-': 0.00}
    return mapping.get(sym)

def encode_strand(df):
    df["target_strand"] = [
        symbol_mapping(sym)
        for sym in df["target_strand"]
    ]
    df["grna_target_strand"] = [
        symbol_mapping(sym)
        for sym in df["grna_target_strand"]
    ]
    return df


def study_mapping(name):
    mapping = {
        'Tsai_circle': 0.00/16.00,
        'Finkelstein': 1.00/16.00,
        'Tsai': 2.00/16.00,
        'Cameron': 3.00/16.00,
        'Kleinstiver': 4.00/16.00,
        'Slaymaker': 5.00/16.00,
        'Kim16': 6.00/16.00,
        'Ran': 7.00/16.00,
        'Anderson': 8.00/16.00,
        'KimChromatin': 9.00/16.00,
        'Chen17': 10.00/16.00,
        'Listgarten': 11.00/16.00,
        'Cho': 12.00/16.00,
        'Kim': 13.00/16.00,
        'Fu': 14.00/16.00,
        'Frock': 15.00/16.00,
        'Wang': 16.00/16.00
    }
    return mapping.get(name)

def encode_study(df):
    df["study_name"] = [
        study_mapping(name)
        for name in df["study_name"]
    ]
    return df
df_X = encode_strand(df_X)
df_X = encode_study(df_X)

#Remove dash
for col in df_X.select_dtypes(exclude = ["number"]).columns:
    df_X[col] = [
        seq.replace("-", "")
        for seq in df_X[col]
    ]

# Pad
for col in df_X.select_dtypes(exclude = ["number"]).columns:
    df_X[col] = df_X[col].str.pad(width = 50, side = "right", fillchar = "X")

# Encode
def encode_nt(nt:str) -> int:
    assert len(nt) == 1
    encoding_dict = {
        'X': [0, 0, 0, 0],
        'A': [1, 0, 0, 0],
        'T': [0, 1, 0, 0],
        'G': [0, 0, 1, 0],
        'C': [0, 0, 0, 1]
    }
    return encoding_dict.get(nt.upper())
def encode_seq(seq:str):
    encoding = [
        encode_nt(nt)
        for nt in seq
    ]
    encoding = np.asarray(encoding).flatten()
    return np.array(encoding)


def encode_col(df, col):
    encoded = [
        encode_seq(seq)
        for seq in df[col]
    ]
    return encoded



def encode(df):
    for col in df.select_dtypes(exclude = ["number"]).columns:
        encode_col(df, col)
    return df
df_X = encode(df_X)
df_X

,Unnamed: 0,target_strand,target_sequence,grna_target_strand,grna_target_sequence,cleavage_freq,energy_1,energy_2,energy_3,energy_4,energy_5,study_name,whole_genome,delivery_mode
0,5077,1.0,ACCCCCCCCAACCCCGCCTCGGCXXXXXXXXXXXXXXXXXXXXXXX...,0.0,GACCCCCTCCACCCCGCCTCCGGXXXXXXXXXXXXXXXXXXXXXXX...,1.560000e-04,26.355,0.000000,8.809075,0.0000,26.355,0.0000,1,0
1,8778,0.0,AAGAGGAGGGAGATTGTTCCTGGXXXXXXXXXXXXXXXXXXXXXXX...,0.0,GGGTGGGGGGAGTTTGCTCCTGGXXXXXXXXXXXXXXXXXXXXXXX...,2.730000e-04,12.735,-4.939260,-4.939260,16.3350,16.335,0.0000,1,0
2,25209,1.0,GTGATAAGTGGAATTGCCATGTGAGXXXXXXXXXXXXXXXXXXXXX...,1.0,GTGATAAGTGGAATGCCATGTGGXXXXXXXXXXXXXXXXXXXXXXX...,9.213702e-06,-12.695,-42.815277,-47.572529,-9.1755,-10.195,0.0625,0,2
3,1750,1.0,AGCACTGTGGATGGAGTTGGAGGXXXXXXXXXXXXXXXXXXXXXXX...,1.0,GGCACTGCGGCTGGAGGTGGGGGXXXXXXXXXXXXXXXXXXXXXXX...,3.302000e-03,17.605,5.982632,5.982632,21.1050,21.105,0.0000,1,0
4,24783,1.0,GTGATAAGATGGAATGCCATGTGGGXXXXXXXXXXXXXXXXXXXXX...,1.0,GTGATAAGTGGAATGCCATGTGGXXXXXXXXXXXXXXXXXXXXXXX...,4.029809e-04,-13.315,-48.090126,-48.090126,-10.8150,-10.815,0.0625,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20500,22598,1.0,GTGGATAAGTGGAACTGCCATGTGGXXXXXXXXXXXXXXXXXXXXX...,1.0,GTGATAAGTGGAATGCCATGTGGXXXXXXXXXXXXXXXXXXXXXXX...,1.935917e-05,-2.095,-26.843519,-26.843519,0.4050,0.405,0.0625,0,2
20501,13984,1.0,GACGCATAAAGATGAGACGCTTCXXXXXXXXXXXXXXXXXXXXXXX...,1.0,GACGCATAAAGATGAGACGCTGGXXXXXXXXXXXXXXXXXXXXXXX...,5.672865e-05,28.700,0.000000,21.484870,0.0000,28.700,0.0625,0,2
20502,22846,1.0,GTGATAAAGTGGAATCGCCATGTGGXXXXXXXXXXXXXXXXXXXXX...,1.0,GTGATAAGTGGAATGCCATGTGGXXXXXXXXXXXXXXXXXXXXXXX...,6.571826e-19,-3.700,-29.040044,-29.040044,-1.2000,-1.200,0.0625,0,2
20503,6433,0.0,TCTCCCCGCCCCCTCGCCTCTGGXXXXXXXXXXXXXXXXXXXXXXX...,0.0,GACCCCCTCCACCCCGCCTCCGGXXXXXXXXXXXXXXXXXXXXXXX...,2.800000e-05,17.935,-6.524068,-6.524068,17.9350,17.935,0.0000,1,0


In [8]:
df_X

,Unnamed: 0,target_strand,target_sequence,grna_target_strand,grna_target_sequence,cleavage_freq,energy_1,energy_2,energy_3,energy_4,energy_5,study_name,whole_genome,delivery_mode
0,5077,1.0,ACCCCCCCCAACCCCGCCTCGGCXXXXXXXXXXXXXXXXXXXXXXX...,0.0,GACCCCCTCCACCCCGCCTCCGGXXXXXXXXXXXXXXXXXXXXXXX...,1.560000e-04,26.355,0.000000,8.809075,0.0000,26.355,0.0000,1,0
1,8778,0.0,AAGAGGAGGGAGATTGTTCCTGGXXXXXXXXXXXXXXXXXXXXXXX...,0.0,GGGTGGGGGGAGTTTGCTCCTGGXXXXXXXXXXXXXXXXXXXXXXX...,2.730000e-04,12.735,-4.939260,-4.939260,16.3350,16.335,0.0000,1,0
2,25209,1.0,GTGATAAGTGGAATTGCCATGTGAGXXXXXXXXXXXXXXXXXXXXX...,1.0,GTGATAAGTGGAATGCCATGTGGXXXXXXXXXXXXXXXXXXXXXXX...,9.213702e-06,-12.695,-42.815277,-47.572529,-9.1755,-10.195,0.0625,0,2
3,1750,1.0,AGCACTGTGGATGGAGTTGGAGGXXXXXXXXXXXXXXXXXXXXXXX...,1.0,GGCACTGCGGCTGGAGGTGGGGGXXXXXXXXXXXXXXXXXXXXXXX...,3.302000e-03,17.605,5.982632,5.982632,21.1050,21.105,0.0000,1,0
4,24783,1.0,GTGATAAGATGGAATGCCATGTGGGXXXXXXXXXXXXXXXXXXXXX...,1.0,GTGATAAGTGGAATGCCATGTGGXXXXXXXXXXXXXXXXXXXXXXX...,4.029809e-04,-13.315,-48.090126,-48.090126,-10.8150,-10.815,0.0625,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20500,22598,1.0,GTGGATAAGTGGAACTGCCATGTGGXXXXXXXXXXXXXXXXXXXXX...,1.0,GTGATAAGTGGAATGCCATGTGGXXXXXXXXXXXXXXXXXXXXXXX...,1.935917e-05,-2.095,-26.843519,-26.843519,0.4050,0.405,0.0625,0,2
20501,13984,1.0,GACGCATAAAGATGAGACGCTTCXXXXXXXXXXXXXXXXXXXXXXX...,1.0,GACGCATAAAGATGAGACGCTGGXXXXXXXXXXXXXXXXXXXXXXX...,5.672865e-05,28.700,0.000000,21.484870,0.0000,28.700,0.0625,0,2
20502,22846,1.0,GTGATAAAGTGGAATCGCCATGTGGXXXXXXXXXXXXXXXXXXXXX...,1.0,GTGATAAGTGGAATGCCATGTGGXXXXXXXXXXXXXXXXXXXXXXX...,6.571826e-19,-3.700,-29.040044,-29.040044,-1.2000,-1.200,0.0625,0,2
20503,6433,0.0,TCTCCCCGCCCCCTCGCCTCTGGXXXXXXXXXXXXXXXXXXXXXXX...,0.0,GACCCCCTCCACCCCGCCTCCGGXXXXXXXXXXXXXXXXXXXXXXX...,2.800000e-05,17.935,-6.524068,-6.524068,17.9350,17.935,0.0000,1,0


In [9]:
target_seq = encode_col(df_X, "target_sequence")

In [10]:
target_seq = np.asarray(target_seq, dtype = np.float32)

In [11]:
target_seq

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [12]:
grna_target_seq = encode_col(df_X, "grna_target_sequence")

In [13]:
grna_target_seq = np.asarray(grna_target_seq, dtype = np.float32)

In [14]:
grna_target_seq

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [15]:
seqs = np.concatenate((target_seq, grna_target_seq), axis = 1)

In [16]:
target_strand = np.asarray(df_X["target_strand"], dtype = np.float32)
target_strand

array([1., 0., 1., ..., 1., 0., 1.], dtype=float32)

In [17]:
grna_target_strand = np.asarray(df_X["grna_target_strand"], dtype = np.float32)
grna_target_strand

array([0., 0., 1., ..., 1., 0., 1.], dtype=float32)

In [18]:
strands = zip(target_strand, grna_target_strand)
strands = tuple(strands)
strands = np.asarray(strands, dtype = np.float32)

In [19]:
e1 = df_X["energy_1"]
e2 = df_X["energy_2"]
e3 = df_X["energy_3"]
e4 = df_X["energy_4"]
e5 = df_X["energy_5"]

energies = zip(e1, e2, e3, e4, e5)
energies = tuple(energies)
energies = np.asarray(energies, dtype = np.float32)

In [20]:
energies.shape

(20450, 5)

In [21]:
study_name = df_X["study_name"]
delivery_mode = df_X["delivery_mode"]
whole_genome = df_X["whole_genome"]

study_details = zip(study_name, delivery_mode, whole_genome)
study_details = tuple(study_details)
study_details = np.asarray(study_details, dtype = np.float32)

In [22]:
strands.dtype

dtype('float32')

In [23]:
all_data = np.concatenate((strands, seqs, energies, study_details), axis = 1)

In [26]:
all_data.dtype

dtype('float32')

In [27]:
import torch
X = torch.from_numpy(all_data)

In [28]:
X

tensor([[1.0000, 0.0000, 1.0000,  ..., 0.0000, 0.0000, 1.0000],
        [0.0000, 0.0000, 1.0000,  ..., 0.0000, 0.0000, 1.0000],
        [1.0000, 1.0000, 0.0000,  ..., 0.0625, 2.0000, 0.0000],
        ...,
        [1.0000, 1.0000, 0.0000,  ..., 0.0625, 2.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.0000],
        [1.0000, 1.0000, 0.0000,  ..., 0.0625, 2.0000, 0.0000]])

In [29]:
y = data["cleavage_freq"]

In [30]:
y = np.array(y).reshape(y.shape[0], 1).astype(np.float32)

In [31]:
y = torch.tensor(y)

In [32]:
y.shape

torch.Size([20450, 1])

In [33]:
X.shape

torch.Size([20450, 410])

In [34]:
input_dim = X.shape[1]

In [36]:
from torch.autograd import Variable

X = Variable(X)
y = Variable(y)

In [38]:
import torch.nn as nn
model = nn.Linear(input_dim, 1)

In [39]:
loss_func = nn.MSELoss()

In [40]:
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [41]:
BATCH_SIZE = 1

In [42]:
EPOCH = 1

In [44]:
import torch.utils.data as Data
torch_dataset = Data.TensorDataset(X, y)

In [45]:
loader = Data.DataLoader(
    dataset = torch_dataset,
    batch_size = BATCH_SIZE,
    shuffle = True,
    num_workers = 2
)

In [47]:
for epoch in range(EPOCH):
    for step, (batch_x, batch_y) in enumerate(loader):
        prediction = model(batch_x)
        print(prediction)

tensor([[-2.6330]], grad_fn=<AddmmBackward>)
tensor([[0.7507]], grad_fn=<AddmmBackward>)
tensor([[1.0990]], grad_fn=<AddmmBackward>)
tensor([[-2.0344]], grad_fn=<AddmmBackward>)
tensor([[1.0444]], grad_fn=<AddmmBackward>)
tensor([[0.5783]], grad_fn=<AddmmBackward>)
tensor([[-2.1382]], grad_fn=<AddmmBackward>)
tensor([[0.2242]], grad_fn=<AddmmBackward>)
tensor([[0.1927]], grad_fn=<AddmmBackward>)
tensor([[0.1105]], grad_fn=<AddmmBackward>)
tensor([[0.0821]], grad_fn=<AddmmBackward>)
tensor([[0.3749]], grad_fn=<AddmmBackward>)
tensor([[-0.1962]], grad_fn=<AddmmBackward>)
tensor([[0.3182]], grad_fn=<AddmmBackward>)
tensor([[-1.6032]], grad_fn=<AddmmBackward>)
tensor([[-1.5507]], grad_fn=<AddmmBackward>)
tensor([[-0.2709]], grad_fn=<AddmmBackward>)
tensor([[0.5855]], grad_fn=<AddmmBackward>)
tensor([[-2.6613]], grad_fn=<AddmmBackward>)
tensor([[0.2498]], grad_fn=<AddmmBackward>)
tensor([[0.1407]], grad_fn=<AddmmBackward>)
tensor([[-0.2340]], grad_fn=<AddmmBackward>)
tensor([[0.1808]], grad

tensor([[0.6898]], grad_fn=<AddmmBackward>)
tensor([[0.3146]], grad_fn=<AddmmBackward>)
tensor([[0.5152]], grad_fn=<AddmmBackward>)
tensor([[-1.2016]], grad_fn=<AddmmBackward>)
tensor([[-2.9789]], grad_fn=<AddmmBackward>)
tensor([[-0.1922]], grad_fn=<AddmmBackward>)
tensor([[-1.2713]], grad_fn=<AddmmBackward>)
tensor([[-1.8499]], grad_fn=<AddmmBackward>)
tensor([[0.2475]], grad_fn=<AddmmBackward>)
tensor([[-1.4583]], grad_fn=<AddmmBackward>)
tensor([[0.3880]], grad_fn=<AddmmBackward>)
tensor([[-0.3049]], grad_fn=<AddmmBackward>)
tensor([[-1.6801]], grad_fn=<AddmmBackward>)
tensor([[0.2546]], grad_fn=<AddmmBackward>)
tensor([[-1.4684]], grad_fn=<AddmmBackward>)
tensor([[0.2964]], grad_fn=<AddmmBackward>)
tensor([[-0.4485]], grad_fn=<AddmmBackward>)
tensor([[-1.8987]], grad_fn=<AddmmBackward>)
tensor([[-0.3105]], grad_fn=<AddmmBackward>)
tensor([[-0.7518]], grad_fn=<AddmmBackward>)
tensor([[-2.1403]], grad_fn=<AddmmBackward>)
tensor([[0.1919]], grad_fn=<AddmmBackward>)
tensor([[-0.9993]]

tensor([[-1.5808]], grad_fn=<AddmmBackward>)
tensor([[0.6486]], grad_fn=<AddmmBackward>)
tensor([[-2.1491]], grad_fn=<AddmmBackward>)
tensor([[0.4068]], grad_fn=<AddmmBackward>)
tensor([[0.0264]], grad_fn=<AddmmBackward>)
tensor([[0.0174]], grad_fn=<AddmmBackward>)
tensor([[-2.2323]], grad_fn=<AddmmBackward>)
tensor([[-0.6173]], grad_fn=<AddmmBackward>)
tensor([[-0.6254]], grad_fn=<AddmmBackward>)
tensor([[0.5605]], grad_fn=<AddmmBackward>)
tensor([[-1.5727]], grad_fn=<AddmmBackward>)
tensor([[0.0461]], grad_fn=<AddmmBackward>)
tensor([[-1.5311]], grad_fn=<AddmmBackward>)
tensor([[0.3897]], grad_fn=<AddmmBackward>)
tensor([[1.5436]], grad_fn=<AddmmBackward>)
tensor([[-1.7672]], grad_fn=<AddmmBackward>)
tensor([[0.5390]], grad_fn=<AddmmBackward>)
tensor([[0.0972]], grad_fn=<AddmmBackward>)
tensor([[0.2897]], grad_fn=<AddmmBackward>)
tensor([[1.1061]], grad_fn=<AddmmBackward>)
tensor([[-1.2894]], grad_fn=<AddmmBackward>)
tensor([[0.0312]], grad_fn=<AddmmBackward>)
tensor([[-0.7786]], gra

tensor([[0.4439]], grad_fn=<AddmmBackward>)
tensor([[-1.9229]], grad_fn=<AddmmBackward>)
tensor([[0.4597]], grad_fn=<AddmmBackward>)
tensor([[-0.3053]], grad_fn=<AddmmBackward>)
tensor([[0.4609]], grad_fn=<AddmmBackward>)
tensor([[-2.4955]], grad_fn=<AddmmBackward>)
tensor([[0.7813]], grad_fn=<AddmmBackward>)
tensor([[-1.1337]], grad_fn=<AddmmBackward>)
tensor([[1.0612]], grad_fn=<AddmmBackward>)
tensor([[0.9287]], grad_fn=<AddmmBackward>)
tensor([[0.8508]], grad_fn=<AddmmBackward>)
tensor([[0.4557]], grad_fn=<AddmmBackward>)
tensor([[0.4011]], grad_fn=<AddmmBackward>)
tensor([[0.6979]], grad_fn=<AddmmBackward>)
tensor([[-0.4533]], grad_fn=<AddmmBackward>)
tensor([[0.6020]], grad_fn=<AddmmBackward>)
tensor([[0.1950]], grad_fn=<AddmmBackward>)
tensor([[0.1138]], grad_fn=<AddmmBackward>)
tensor([[0.4677]], grad_fn=<AddmmBackward>)
tensor([[-2.2548]], grad_fn=<AddmmBackward>)
tensor([[1.0689]], grad_fn=<AddmmBackward>)
tensor([[-1.5741]], grad_fn=<AddmmBackward>)
tensor([[1.1007]], grad_f

tensor([[0.2646]], grad_fn=<AddmmBackward>)
tensor([[-0.5219]], grad_fn=<AddmmBackward>)
tensor([[0.5804]], grad_fn=<AddmmBackward>)
tensor([[0.6973]], grad_fn=<AddmmBackward>)
tensor([[-0.4357]], grad_fn=<AddmmBackward>)
tensor([[-2.0050]], grad_fn=<AddmmBackward>)
tensor([[-0.0951]], grad_fn=<AddmmBackward>)
tensor([[0.0996]], grad_fn=<AddmmBackward>)
tensor([[-2.5665]], grad_fn=<AddmmBackward>)
tensor([[0.7252]], grad_fn=<AddmmBackward>)
tensor([[-0.1686]], grad_fn=<AddmmBackward>)
tensor([[0.7554]], grad_fn=<AddmmBackward>)
tensor([[-2.4151]], grad_fn=<AddmmBackward>)
tensor([[-1.5722]], grad_fn=<AddmmBackward>)
tensor([[0.8831]], grad_fn=<AddmmBackward>)
tensor([[-1.9566]], grad_fn=<AddmmBackward>)
tensor([[-2.5585]], grad_fn=<AddmmBackward>)
tensor([[-0.6247]], grad_fn=<AddmmBackward>)
tensor([[-0.4591]], grad_fn=<AddmmBackward>)
tensor([[-0.0395]], grad_fn=<AddmmBackward>)
tensor([[-0.3231]], grad_fn=<AddmmBackward>)
tensor([[0.7181]], grad_fn=<AddmmBackward>)
tensor([[-0.6046]]

...

tensor([[0.5254]], grad_fn=<AddmmBackward>)
tensor([[0.3343]], grad_fn=<AddmmBackward>)
tensor([[-1.1326]], grad_fn=<AddmmBackward>)
tensor([[-0.3205]], grad_fn=<AddmmBackward>)
tensor([[-0.2969]], grad_fn=<AddmmBackward>)
tensor([[0.1590]], grad_fn=<AddmmBackward>)
tensor([[0.4303]], grad_fn=<AddmmBackward>)
tensor([[-0.2879]], grad_fn=<AddmmBackward>)
tensor([[0.2961]], grad_fn=<AddmmBackward>)
tensor([[-0.4051]], grad_fn=<AddmmBackward>)
tensor([[-0.2764]], grad_fn=<AddmmBackward>)
tensor([[0.4658]], grad_fn=<AddmmBackward>)
tensor([[0.8574]], grad_fn=<AddmmBackward>)
tensor([[-2.0537]], grad_fn=<AddmmBackward>)
tensor([[-1.1771]], grad_fn=<AddmmBackward>)
tensor([[-1.8851]], grad_fn=<AddmmBackward>)
tensor([[0.0618]], grad_fn=<AddmmBackward>)
tensor([[-2.7848]], grad_fn=<AddmmBackward>)
tensor([[0.3541]], grad_fn=<AddmmBackward>)
tensor([[0.7442]], grad_fn=<AddmmBackward>)
tensor([[0.2003]], grad_fn=<AddmmBackward>)
tensor([[0.2321]], grad_fn=<AddmmBackward>)
tensor([[-2.2416]], gr

tensor([[1.5153]], grad_fn=<AddmmBackward>)
tensor([[0.2265]], grad_fn=<AddmmBackward>)
tensor([[0.7529]], grad_fn=<AddmmBackward>)
tensor([[-1.1901]], grad_fn=<AddmmBackward>)
tensor([[1.0011]], grad_fn=<AddmmBackward>)
tensor([[0.0733]], grad_fn=<AddmmBackward>)
tensor([[0.8317]], grad_fn=<AddmmBackward>)
tensor([[-0.2034]], grad_fn=<AddmmBackward>)
tensor([[0.2877]], grad_fn=<AddmmBackward>)
tensor([[-0.2469]], grad_fn=<AddmmBackward>)
tensor([[-2.8015]], grad_fn=<AddmmBackward>)
tensor([[1.0151]], grad_fn=<AddmmBackward>)
tensor([[0.0557]], grad_fn=<AddmmBackward>)
tensor([[-1.4369]], grad_fn=<AddmmBackward>)
tensor([[-1.6037]], grad_fn=<AddmmBackward>)
tensor([[-1.4112]], grad_fn=<AddmmBackward>)
tensor([[1.1184]], grad_fn=<AddmmBackward>)
tensor([[-2.8042]], grad_fn=<AddmmBackward>)
tensor([[0.2244]], grad_fn=<AddmmBackward>)
tensor([[1.4916]], grad_fn=<AddmmBackward>)
tensor([[-2.9862]], grad_fn=<AddmmBackward>)
tensor([[-2.2227]], grad_fn=<AddmmBackward>)
tensor([[-0.9217]], gr

tensor([[-2.7858]], grad_fn=<AddmmBackward>)
tensor([[0.0587]], grad_fn=<AddmmBackward>)
tensor([[-0.3882]], grad_fn=<AddmmBackward>)
tensor([[-1.9733]], grad_fn=<AddmmBackward>)
tensor([[0.4030]], grad_fn=<AddmmBackward>)
tensor([[0.1733]], grad_fn=<AddmmBackward>)
tensor([[0.6654]], grad_fn=<AddmmBackward>)
tensor([[-0.5902]], grad_fn=<AddmmBackward>)
tensor([[0.0891]], grad_fn=<AddmmBackward>)
tensor([[0.2696]], grad_fn=<AddmmBackward>)
tensor([[-2.0664]], grad_fn=<AddmmBackward>)
tensor([[0.3385]], grad_fn=<AddmmBackward>)
tensor([[-2.0704]], grad_fn=<AddmmBackward>)
tensor([[0.7161]], grad_fn=<AddmmBackward>)
tensor([[-1.4809]], grad_fn=<AddmmBackward>)
tensor([[-0.3235]], grad_fn=<AddmmBackward>)
tensor([[-0.7154]], grad_fn=<AddmmBackward>)
tensor([[0.5359]], grad_fn=<AddmmBackward>)
tensor([[-0.1850]], grad_fn=<AddmmBackward>)
tensor([[0.0049]], grad_fn=<AddmmBackward>)
tensor([[-2.0563]], grad_fn=<AddmmBackward>)
tensor([[0.1625]], grad_fn=<AddmmBackward>)
tensor([[0.4098]], gr

tensor([[-2.1738]], grad_fn=<AddmmBackward>)
tensor([[0.7949]], grad_fn=<AddmmBackward>)
tensor([[-1.5263]], grad_fn=<AddmmBackward>)
tensor([[-2.6181]], grad_fn=<AddmmBackward>)
tensor([[0.3347]], grad_fn=<AddmmBackward>)
tensor([[-0.0747]], grad_fn=<AddmmBackward>)
tensor([[-0.3273]], grad_fn=<AddmmBackward>)
tensor([[0.7845]], grad_fn=<AddmmBackward>)
tensor([[0.4551]], grad_fn=<AddmmBackward>)
tensor([[0.0638]], grad_fn=<AddmmBackward>)
tensor([[0.3721]], grad_fn=<AddmmBackward>)
tensor([[0.0029]], grad_fn=<AddmmBackward>)
tensor([[0.2018]], grad_fn=<AddmmBackward>)
tensor([[-2.7679]], grad_fn=<AddmmBackward>)
tensor([[0.5511]], grad_fn=<AddmmBackward>)
tensor([[0.7202]], grad_fn=<AddmmBackward>)
tensor([[0.9480]], grad_fn=<AddmmBackward>)
tensor([[-2.8791]], grad_fn=<AddmmBackward>)
tensor([[1.0423]], grad_fn=<AddmmBackward>)
tensor([[0.5050]], grad_fn=<AddmmBackward>)
tensor([[0.5950]], grad_fn=<AddmmBackward>)
tensor([[0.3460]], grad_fn=<AddmmBackward>)
tensor([[0.3976]], grad_f

tensor([[-0.0444]], grad_fn=<AddmmBackward>)
tensor([[0.4347]], grad_fn=<AddmmBackward>)
tensor([[-1.6160]], grad_fn=<AddmmBackward>)
tensor([[0.2212]], grad_fn=<AddmmBackward>)
tensor([[0.2306]], grad_fn=<AddmmBackward>)
tensor([[-0.4292]], grad_fn=<AddmmBackward>)
tensor([[-0.3827]], grad_fn=<AddmmBackward>)
tensor([[-0.0237]], grad_fn=<AddmmBackward>)
tensor([[0.4253]], grad_fn=<AddmmBackward>)
tensor([[0.7337]], grad_fn=<AddmmBackward>)
tensor([[-0.4738]], grad_fn=<AddmmBackward>)
tensor([[0.2401]], grad_fn=<AddmmBackward>)
tensor([[0.2419]], grad_fn=<AddmmBackward>)
tensor([[0.3664]], grad_fn=<AddmmBackward>)
tensor([[-0.0802]], grad_fn=<AddmmBackward>)
tensor([[0.4647]], grad_fn=<AddmmBackward>)
tensor([[0.8318]], grad_fn=<AddmmBackward>)
tensor([[0.2557]], grad_fn=<AddmmBackward>)
tensor([[-1.8105]], grad_fn=<AddmmBackward>)
tensor([[0.6581]], grad_fn=<AddmmBackward>)
tensor([[0.4378]], grad_fn=<AddmmBackward>)
tensor([[0.4834]], grad_fn=<AddmmBackward>)
tensor([[0.4723]], grad_

In [ ]:
chicken

In [ ]:
df_X.dtypes

In [ ]:
df_X

In [ ]:
tuple((df_X["target_sequence"][0]))

In [ ]:
def fold_seq(seq):
    return sum(seq, [])
seq = [[1, 2],[3, 4]]
fold_seq(seq)

In [ ]:
seq[1]

In [ ]:
np.concatenate((seq[0], seq[1]), axis = None)

In [ ]:
from functools import reduce

def concat_nt(first, second):
    return np.concatenate((first, second), axis = None)

def concat_seq(seq):
    return np.asarray(reduce(concat_nt, seq))
    
def concat(df):
    df["grna_target_sequence"] = [
        concat_seq(seq)
        for seq in df["grna_target_sequence"]
    ]
    df["target_sequence"] = [
        concat_seq(seq)
        for seq in df["target_sequence"]
    ]
    return df

concat(df_X)
    

In [ ]:
strands = np.asarray(tuple(zip(df_X["target_strand"], df_X["grna_target_strand"])))
strands

In [ ]:
target_seq = df_X["target_sequence"]
target_seq.shape

In [ ]:
target_seq

In [ ]:
chicken

In [ ]:
def get_strands(df):
    target_strand = df["target_strand"]
    grna_target_strand = df["grna_target_strand"]
    target_seq = df["target_sequence"]
    target_seq = np.asarray(target_seq)
    print(target_seq)
    grna_target_seq = df["grna_target_sequence"]
    seqs = np.vstack((target_seq, grna_target_seq))
    print(seqs)
    
    strands = zip(target_strand, grna_target_strand)
    strands = tuple(strands)
    strands = np.asarray(strands)
    print(strands)

In [ ]:
get_strands(df_X)

In [ ]:
df_X["target"] = np.asarray(df_X["target_sequence"].apply(lambda x: x.tolist()) + df_X["grna_target_sequence"].apply(lambda x: x.tolist()))

In [ ]:
x = np.asarray(df_X["target"][0])
x

In [ ]:
y = np.array([1, 2, 3, 4])

In [ ]:
np.asarray(x.tolist() + y.tolist())

In [ ]:
def get_strands(df, row):
    return np.array([df["target_strand"][row], df["grna_target_strand"][row]])

def get_target_seq(df, row):
    return df["target_sequence"][row]
    
def get_grna_target_seq(df, row):
    return df["grna_target_sequence"][row]

def get_energies(df, row):
    return np.array([df_X["energy_1"][row], df_X["energy_2"][row], df_X["energy_3"][row], df_X["energy_4"][row], df_X["energy_5"][row]])

def get_study_details(df, row):
    return np.array([df_X["study_name"][row], df_X["whole_genome"][row], df_X["delivery_mode"][row]])

def concat(df):
    

In [ ]:
f = np.array([df_X["target_strand"][0], df_X["grna_target_strand"][0]])
f

In [ ]:
g = df_X["target_sequence"][0]
g

In [ ]:
h = df_X["grna_target_sequence"][0]

In [ ]:
i = np.array([df_X["energy_1"][0], df_X["energy_2"][0], df_X["energy_3"][0], df_X["energy_4"][0], df_X["energy_5"][0]])
i

In [ ]:
j = np.array([df_X["study_name"][0], df_X["whole_genome"][0], df_X["delivery_mode"][0]])
j

In [ ]:
np.concatenate((f, g, h, i, j), axis = None)

In [ ]:
import torch
import torch.nn as nn

input = torch.randn(10, 2)
output = torch.flatten(input)

In [ ]:
output

In [ ]:
import functools

def fold(df):
    df["stacked"] = functools.reduce(lambda x, y: df[x].apply(lambda x: x.tolist()) + df[y].apply(lambda x: x.tolist()), df.columns)
    return df

In [ ]:
df_X["stacked"] = df_X["target_strand"]

In [ ]:
df_X["target_strand"].to_numpy().shape

In [ ]:
df_X["grna_target_strand"].to_numpy().shape

In [ ]:
catted = np.vstack((df_X["target_strand"].to_numpy(), 
                    df_X["grna_target_strand"].to_numpy(), 
                    df_X["grna_target_sequence"].to_numpy(), 
                    df_X["target_sequence"].to_numpy(),
                    df_X["energy_1"].to_numpy(),
                    df_X["energy_2"].to_numpy(),
                    df_X["energy_3"].to_numpy(),
                    df_X["energy_4"].to_numpy(),
                    df_X["energy_5"].to_numpy(),
                    df_X["study_name"].to_numpy(),
                    df_X["whole_genome"].to_numpy(),
                    df_X["delivery_mode"].to_numpy()
                   )).T

In [ ]:
catted.shape

In [ ]:
c = catted[0].flatten()

In [ ]:
chicken

In [ ]:
X = df_X

In [ ]:
y = data["cleavage_freq"]
y

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model = ExtraTreesClassifier()
model.fit(X,y)
print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()